In [1]:
import sys
import os

In [2]:
import numpy as np
import pandas as pd

In [3]:
taggram = np.load("MusicCNNFeatures/taggram_array.npy")

In [4]:
print(np.shape(taggram))

(400, 20, 50)


In [5]:
label_songs = pd.read_csv("./preprocessing/labels.csv")
info_songs = pd.read_csv("./Info/info.csv")

select_label = "genre"

if select_label == "genre":
    labels = label_songs[select_label].map({"classical":0, "electronic":1, "pop":2, "rock":3})
labels = labels.to_numpy()

In [6]:
n_data = len(label_songs)
idx = np.random.permutation(n_data)
info_songs = info_songs.reindex(idx)
taggram = taggram[idx, :, :]
labels = labels[idx]

In [7]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Input, layers
from tensorflow.keras import backend as K

sample_size = (5, 50)

drop_out_rate = 0.2

# very simple keras Sequential model
input_tensor = Input(sample_size)
x = layers.Dense(50, activation="relu")(input_tensor)
x = layers.Flatten()(x)
x = layers.Dropout(drop_out_rate)(x)
output_tensor = layers.Dense(4, activation="softmax")(x)

model_1 = tf.keras.Model(input_tensor, output_tensor)

In [8]:
model_1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5, 50)]           0         
_________________________________________________________________
dense (Dense)                (None, 5, 50)             2550      
_________________________________________________________________
flatten (Flatten)            (None, 250)               0         
_________________________________________________________________
dropout (Dropout)            (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1004      
Total params: 3,554
Trainable params: 3,554
Non-trainable params: 0
_________________________________________________________________


In [9]:
def slice_accordingly(input_tensor, labels, size=5):

    input_shape = (tf.shape(input_tensor).numpy())
    
    input_shape[1] = size
    size = input_shape
    
    sliced_tensor = tf.image.random_crop(input_tensor, size)
    
    nan_values, idx = tf.unique(tf.gather(tf.where(tf.math.is_nan(sliced_tensor)), 0, axis=1))
    nan_values = nan_values.numpy()
  
    msk = np.zeros((input_shape[0]), dtype=np.bool)
    msk[nan_values] = True
    msk = ~msk

    sliced_tensor = tf.boolean_mask(sliced_tensor, msk , axis=0)
    labels = tf.boolean_mask(labels , msk, axis=0)
    
    return sliced_tensor, labels

In [10]:
fraction_validation = 0.5
n_test = int(fraction_validation*len(labels))
n_train = len(labels) - n_test

train_features, train_labels = taggram[:n_train], labels[:n_train]
validation_features, validation_labels = taggram[n_train:], labels[n_train:]

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_features, validation_labels))

In [12]:
model = model_1

model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

n_epochs = 100
batch_size = 50

for epoch in range(n_epochs):
    print("Epoch", epoch)
    train_ds = train_dataset.shuffle(n_train).batch(batch_size)
    val_ds = validation_dataset.shuffle(n_test).batch(n_test)
    
    x_val, y_val = next(iter(val_ds))
    x_val, y_val = slice_accordingly(x_val, y_val)
    
    for features, labels in train_ds:
        features, labels = slice_accordingly(features, labels)
        
        history = model.fit(features, labels,
                           validation_data=(x_val, y_val),
                           verbose=1) 
        predictions = model.predict(features)

Epoch 0
2/2 [==============================] - 0s 71ms/step - loss: 1.3326 - accuracy: 0.4200 - val_loss: 1.3335 - val_accuracy: 0.3858
Epoch 1
2/2 [==============================] - 0s 71ms/step - loss: 1.2857 - accuracy: 0.4898 - val_loss: 1.2590 - val_accuracy: 0.6400
Epoch 2
2/2 [==============================] - 0s 68ms/step - loss: 1.1689 - accuracy: 0.8163 - val_loss: 1.1843 - val_accuracy: 0.6950
Epoch 3
2/2 [==============================] - 0s 70ms/step - loss: 1.1609 - accuracy: 0.6200 - val_loss: 1.1025 - val_accuracy: 0.7150
Epoch 4
2/2 [==============================] - 0s 72ms/step - loss: 1.0545 - accuracy: 0.6875 - val_loss: 1.0086 - val_accuracy: 0.7350
Epoch 5
2/2 [==============================] - 0s 70ms/step - loss: 0.8867 - accuracy: 0.7600 - val_loss: 0.9107 - val_accuracy: 0.7462
Epoch 6
2/2 [==============================] - 0s 69ms/step - loss: 0.7936 - accuracy: 0.7200 - val_loss: 0.8269 - val_accuracy: 0.7286
Epoch 7
2/2 [==============================] - 0

KeyboardInterrupt: 